# Import library for cropping the images

In [1]:
import cv2
import dlib
import tkinter as tk 
from tkinter import filedialog 

# To access get_frontal_face_detector class to access frontal face of a person

In [2]:
detector = dlib.get_frontal_face_detector()

# Path for saving all cropped images

In [3]:
new_path ='E:\\Departmental_Project_College\\Cropped Image\\pic '

# To draw stylish rectangle around the objects

In [4]:
def MyRec(rgb,x,y,w,h,v=20,color=(200,0,0),thikness =2):
    #cv2.line(image, start_point, end_point, color, thickness)
    #Left-Top Corner
    cv2.line(rgb, (x,y),(x+v,y), color, thikness)
    cv2.line(rgb, (x,y),(x,y+v), color, thikness)
    #right-Top corner
    cv2.line(rgb, (x+w,y),(x+w-v,y), color, thikness)
    cv2.line(rgb, (x+w,y),(x+w,y+v), color, thikness)
    #Left-Down corner
    cv2.line(rgb, (x,y+h),(x,y+h-v), color, thikness)
    cv2.line(rgb, (x,y+h),(x+v,y+h), color, thikness)
    #right-Down corner
    cv2.line(rgb, (x+w,y+h),(x+w,y+h-v), color, thikness)
    cv2.line(rgb, (x+w,y+h),(x+w-v,y+h), color, thikness)

# To save the images

In [5]:
def save(img,i,name, bbox, width=180,height=227):
    i=str(i)
    x, y, w, h = bbox
    imgCrop = img[y:h, x: w]
    if imgCrop is None:
        print("Wrong Path")
        return
    imgCrop = cv2.resize(imgCrop, (width, height))#we need this line to reshape the images
    cv2.imwrite(name+i+".jpg", imgCrop)


# Face Detection Function

In [6]:
def faces(path_file,i):
    frame =cv2.imread(path_file)
    #Below line is use for Gray image
    #gray =cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = detector(frame)
    # detect the face
    for counter,face in enumerate(faces):
        print(counter)
        x1, y1 = face.left(), face.top()
        x2, y2 = face.right(), face.bottom()
        cv2.rectangle(frame,(x1,y1),(x2,y2),(220,255,220),1)
        MyRec(frame, x1, y1, x2 - x1, y2 - y1, 10, (0,250,0), 3)
        #save(gray,new_path+str(counter),(x1,y1,x2,y2))
        save(frame,i,new_path+str(counter),(x1,y1,x2,y2))
    frame = cv2.resize(frame,(800,800))
    cv2.imshow('img',frame)
    cv2.waitKey(0)
    print("done saving")

In [7]:
n=int(input("Enter total number of picture:-"))
for i in range(n):
    root = tk.Tk() 
    root.withdraw() 
    file_path = filedialog.askopenfilename()
    x=file_path.replace("/","\\")
    faces(x,i)

Enter total number of picture:-5
0
done saving
0
1
2
3
4
5
done saving
0
done saving
0
1
2
3
4
5
6
7
8
done saving
0
done saving


# Make Image DataBase

# Library import

In [8]:
import mysql.connector

In [9]:
MyDB=mysql.connector.connect(
    host="localhost",
    user="root",
    password="Bally@2022",
)

In [10]:
MyCursor=MyDB.cursor()
MyCursor.execute("show databases")
lst=MyCursor.fetchall()
data_name=input("enter database name ")
if(data_name,)in lst:
    print("exist DataBase")
else:
    MyCursor.execute("CREATE DATABASE {}".format(data_name))
    print("DataBase Created")

enter database name db2
DataBase Created


In [11]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Bally@2022",
  database=data_name
)

mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE IF NOT EXISTS Image (id INTEGER(45) NOT NULL AUTO_INCREMENT PRIMARY KEY,Photo LONGBLOB NOT NULL)")

In [12]:
def convertToBinaryData(filename):
    # Convert digital data to binary format
    with open(filename, 'rb') as file:
        binaryData = file.read()
    return binaryData

In [13]:
def insertBLOB(id,photo):
    print("Inserting BLOB into python_employee table")
    try:
        connection = mysql.connector.connect(host='localhost',
                                             database=data_name,
                                             user='root',
                                             password='Bally@2022')

        cursor = connection.cursor()
        sql_insert_blob_query = """ INSERT INTO Image(id,photo) VALUES (%s,%s)"""

        empPicture = convertToBinaryData(photo)
        # Convert data into tuple format
        insert_blob_tuple = (id,empPicture)
        result = cursor.execute(sql_insert_blob_query, insert_blob_tuple)
        connection.commit()
        print("Image and file inserted successfully as a BLOB into python_employee table", result)

    except mysql.connector.Error as error:
        print("Failed inserting BLOB data into MySQL table {}".format(error))

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [16]:
n=int(input("No of picture you want to put in your database:-"))

No of picture you want to put in your database:-2


In [17]:
def wantToInsert():
    for i in range(1,n+1):
        root = tk.Tk() 
        root.withdraw() 
        file_path = filedialog.askopenfilename()
        x=file_path.replace("/","\\")
        insertBLOB(i,x)
    

In [18]:
wantToInsert()

Inserting BLOB into python_employee table
Image and file inserted successfully as a BLOB into python_employee table None
MySQL connection is closed
Inserting BLOB into python_employee table
Image and file inserted successfully as a BLOB into python_employee table None
MySQL connection is closed


In [17]:
def write_file(data, filename):
    # Convert binary data to proper format and write it on Hard Disk
    with open(filename, 'wb') as file:
        file.write(data)

In [18]:
def readBLOB(id, photo):
    print("Reading BLOB data from python_employee table")

    try:
        connection = mysql.connector.connect(host='localhost',
                                             database=data_name,
                                             user='root',
                                             password='Bally@2022')

        cursor = connection.cursor()
        sql_fetch_blob_query = """SELECT * from Image where id = %s"""

        cursor.execute(sql_fetch_blob_query, (id,))
        record = cursor.fetchall()
        for row in record:
            print("Id = ", row[0], )
            image = row[1]
            print("Storing employee image and bio-data on disk \n")
            write_file(image, photo)

    except mysql.connector.Error as error:
        print("Failed to read BLOB data from MySQL table {}".format(error))

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [19]:
stored_path ='E:\\Departmental_Project_College\\Stored_From_Data_Base\\New Bitmap Image.bmp '

In [20]:
def wantToExtract():
    for i in range(1,n+1):
        readBLOB(i,stored_path)

In [21]:
wantToExtract()


Reading BLOB data from python_employee table
Id =  1
Storing employee image and bio-data on disk 

MySQL connection is closed
Reading BLOB data from python_employee table
Id =  2
Storing employee image and bio-data on disk 

MySQL connection is closed
Reading BLOB data from python_employee table
Id =  3
Storing employee image and bio-data on disk 

MySQL connection is closed
Reading BLOB data from python_employee table
Id =  4
Storing employee image and bio-data on disk 

MySQL connection is closed
